# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
 cities = pd.read_csv("city_df.csv", encoding="utf-8")
cities.head()

,Unnamed: 0,city,lat,long,temp,humidity,wind,cloudiness,country,date
0,0,saint george,37.10,-113.58,66.00,36,21.92,1,US,1586558716
1,1,puerto ayora,-0.74,-90.35,84.20,70,6.93,40,EC,1586558748
2,2,port alfred,-33.59,26.89,64.76,86,4.05,49,ZA,1586558507
3,3,pevek,69.70,170.31,6.08,88,22.95,100,RU,1586558508
4,4,leopold,-38.18,144.47,57.00,93,18.34,90,AU,1586558749


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
humidity = cities["humidity"].astype(float)
max_humidity = humidity.max()
locations = cities[["lat", "long"]]


In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrow_city = cities.loc[(cities["temp"] > 70) & (cities["temp"] < 80) & (cities["cloudiness"] == 0), :]
narrow_city = narrow_city.dropna(how='any')
narrow_city.reset_index(inplace=True)
del narrow_city['index']
narrow_city.head()

,Unnamed: 0,city,lat,long,temp,humidity,wind,cloudiness,country,date
0,271,nouadhibou,20.93,-17.03,71.60,64,19.46,0,MR,1586558787
1,296,tura,25.52,90.22,70.88,48,2.95,0,IN,1586558543
2,432,port hedland,-20.32,118.57,71.60,88,5.82,0,AU,1586558816
3,509,cockburn town,21.46,-71.14,79.02,71,2.59,0,TC,1586558543
4,562,san juan,-31.54,-68.54,71.83,33,2.39,0,AR,1586558832


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = []

for i in range(len(narrow_city)):
    lat = narrow_city.loc[i]['lat']
    long = narrow_city.loc[i]['long']

    params = { "location": f"{lat},{long}",
               "radius": 5000,
               "types" : "hotel",
               "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
narrow_city["Hotel Name"] = hotel_df
narrow_city= narrow_city.dropna(how='any')
narrow_city.head()

,Unnamed: 0,city,lat,long,temp,humidity,wind,cloudiness,country,date,Hotel Name
0,271,nouadhibou,20.93,-17.03,71.60,64,19.46,0,MR,1586558787,Nouadhibou
1,296,tura,25.52,90.22,70.88,48,2.95,0,IN,1586558543,Tura
2,432,port hedland,-20.32,118.57,71.60,88,5.82,0,AU,1586558816,Port Hedland
3,509,cockburn town,21.46,-71.14,79.02,71,2.59,0,TC,1586558543,Cockburn Town
4,562,san juan,-31.54,-68.54,71.83,33,2.39,0,AR,1586558832,San Juan


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in narrow_city.iterrows()]
locations = narrow_city[["lat", "long"]]

In [ ]:
# Add marker layer ontop of heatmap
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig